# Crypto_Scraper

## Import 

In [28]:
import praw
import pandas as pd
from datetime import datetime

In [29]:
# Authentifizierungsdetails
reddit = praw.Reddit(
    client_id='Qwk8MQZQ8YxP0iylZOcAzw',           # Deine Client ID
    client_secret='arDMtC8hLTUmWnRe1POda4W-xqj3aw',  # Dein Client Secret
    user_agent='SP3-3 Crypto Scraper/1.0 by le_remis' # User Agent
)

# Subreddit auswählen
subreddit_name = "cryptocurrency"  # Du kannst hier auch andere Subreddits angeben
subreddit = reddit.subreddit(subreddit_name)

In [30]:
print(f"Client ID: {reddit.config.client_id}")
print(f"Client Secret: {reddit.config.client_secret}")
print(f"User Agent: {reddit.config.user_agent}")


Client ID: Qwk8MQZQ8YxP0iylZOcAzw
Client Secret: arDMtC8hLTUmWnRe1POda4W-xqj3aw
User Agent: SP3-3 Crypto Scraper/1.0 by le_remis


In [31]:
try:
    subreddit = reddit.subreddit("CryptoCurrency")
    for post in subreddit.hot(limit=5):
        print(f"Title: {post.title}, Score: {post.score}")
except Exception as e:
    print(f"Fehler beim Abrufen der Subreddit-Daten: {e}")


Title: Daily Crypto Discussion - January 28, 2025 (GMT+0), Score: 28
Title: The half brother of Barack Obama, Malik Obama just launched and rugged a token called "Obama" on Solana., Score: 6211
Title: Buy the dip!, Score: 1487
Title: Bitcoin lover Scott Bessent confirmed as US Treasury Secretary, Score: 654
Title: Ethereum Creator Vitalik Buterin Criticizing Politician-Issued Coins as 'Perfect Bribery Vehicle', Score: 1805


## Liste der Cryptos

In [32]:
cryptos = [
    ("Ethereum", "ETH"),
    ("Ripple", "XRP"),
    ("Tether USD", "USDT"),
    ("Solana", "SOL")
]



In [33]:
subreddits = [
    "CryptoCurrency",  # Allgemeine Diskussionen über Kryptowährungen
    "CryptoMarkets",   # Diskussionen über den Kryptomarkt und Preisbewegungen
    "CryptoTrading",   # Fokus auf Trading-Strategien und Analysen
    "Altcoin",         # Diskussionen über Altcoins (alle Kryptowährungen außer Bitcoin)
    "DeFi",            # Decentralized Finance (DeFi) und Projekte
    "BitcoinBeginners",# Für Anfänger in der Krypto-Welt
    "cryptotechnology", # Fokus auf die zugrunde liegende Blockchain-Technologie
    "cryptocurrencies", # Allgemeine Diskussionen über Kryptowährungen
    "Satoshistreetsbets", # Krypto-Wetten und Spekulationen
    "Binance"        # Diskussionen über die Binance-Plattform  
]

## Scrape

In [34]:
# Funktion, um Posts und Kommentare zu scrapen
def scrape_reddit(start_timestamp, end_timestamp, mode="initial"):
    posts = []
    comments = []
    post_ids = set()  # Set zum Speichern der Post-IDs

    for crypto_name, crypto_symbol in cryptos:
        for subreddit_name in subreddits:
            subreddit = reddit.subreddit(subreddit_name)
            print(f"Suche Posts zu {crypto_name} ({crypto_symbol}) in r/{subreddit_name}...")

            # Suche nach Namen und Kürzel
            for search_term in [crypto_name, crypto_symbol]:
                for post in subreddit.search(query=search_term, sort="new", limit=None):
                    if start_timestamp <= post.created_utc <= end_timestamp and post.id not in post_ids:
                        post_ids.add(post.id)
                        posts.append({
                            'crypto': crypto_name,
                            'search_term': search_term,
                            'subreddit': subreddit_name,
                            'post_id': post.id,
                            'title': post.title,
                            'author': str(post.author),
                            'created_utc': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                            'score': post.score,
                            'num_comments': post.num_comments,
                            'selftext': post.selftext
                        })
                        print(f"Gefunden in r/{subreddit_name}: {post.title} (Suchbegriff: {search_term})")

                        # Kommentare sammeln
                        post.comments.replace_more(limit=0)
                        for comment in post.comments.list():
                            comments.append({
                                'post_id': post.id,
                                'comment_id': comment.id,
                                'author': str(comment.author),
                                'created_utc': datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                                'score': comment.score,
                                'body': comment.body
                            })

    # In DataFrames umwandeln
    df_posts = pd.DataFrame(posts)
    df_comments = pd.DataFrame(comments)

    # DataFrames zurückgeben
    print(f"Scrape abgeschlossen: {len(df_posts)} Posts und {len(df_comments)} Kommentare gefunden.")
    return df_posts, df_comments

# **Einmaliger Scrape für das gesamte Jahr 2025**
start_of_year = int(datetime(2025, 1, 1).timestamp())  # 1. Januar 2025
now = int(datetime.now().timestamp())  # Aktuelle Zeit
print("Starte den einmaligen Scrape für das Jahr 2025...")
df_posts_initial, df_comments_initial = scrape_reddit(start_of_year, now, mode="initial")

# **Wöchentlicher Scrape**
last_week = now - 7 * 24 * 60 * 60  # 7 Tage zurück
print("Starte den wöchentlichen Scrape...")
df_posts_weekly, df_comments_weekly = scrape_reddit(last_week, now, mode="weekly")

# Beispiel: Lokale Weiterverarbeitung
print("Daten können jetzt in der Pipeline bereinigt werden...")

Starte den einmaligen Scrape für das Jahr 2025...
Suche Posts zu Ethereum (ETH) in r/CryptoCurrency...
Gefunden in r/CryptoCurrency: Ethereum Foundation Sells Another 100 ETH, But There's Still 'Hopium' For Holders (Suchbegriff: Ethereum)
Gefunden in r/CryptoCurrency: Bullish Signal? Ethereum Sees Highest Surge in Network Growth Since October 2022 (Suchbegriff: Ethereum)
Gefunden in r/CryptoCurrency: Ethereum developers pledge cooperation as Ether’s woes trigger ‘wartime’ mindset (Suchbegriff: Ethereum)
Gefunden in r/CryptoCurrency: The price of Ethereum (ETH) is testing the support level of $3,000 - CoinKaira (Suchbegriff: Ethereum)
Gefunden in r/CryptoCurrency: Ethereum Creator Vitalik Buterin Criticizing Politician-Issued Coins as 'Perfect Bribery Vehicle' (Suchbegriff: Ethereum)
Gefunden in r/CryptoCurrency: Ethereum Foundation faces pressure as it continues 100 ETH sales amid market drops (Suchbegriff: Ethereum)
Gefunden in r/CryptoCurrency: They call it Deepseek because everyone 

## Datenbereinigung 

In [ ]:
def clean_data(df_posts, df_comments):
    # 1. Duplikate entfernen
    df_posts = df_posts.drop_duplicates(subset=["post_id"])
    df_comments = df_comments.drop_duplicates(subset=["comment_id"])
    
    # 2. Fehlende Werte behandeln
    df_posts['selftext'] = df_posts['selftext'].fillna('')  # Fehlende Posttexte auffüllen
    df_comments['body'] = df_comments['body'].fillna('')  # Fehlende Kommentare auffüllen
    
    # 3. Zeitstempel konvertieren
    df_posts['created_utc'] = pd.to_datetime(df_posts['created_utc'])
    df_comments['created_utc'] = pd.to_datetime(df_comments['created_utc'])
    
    # 4. Optionale Filterung nach Qualität (z. B. Spam entfernen)
    df_posts = df_posts[df_posts['score'] > 0]  # Posts mit negativem Score entfernen
    df_comments = df_comments[df_comments['score'] > 0]  # Kommentare mit negativem Score entfernen
    
    return df_posts, df_comments


Funktion zur bereinigung auf die Daten anwenden

In [ ]:
# Bereinigen der Daten
df_posts_clean, df_comments_clean = clean_data(df_posts_initial, df_comments_initial)

# Überprüfen, wie viele Einträge übrig sind
print(f"Bereinigte Posts: {len(df_posts_clean)}")
print(f"Bereinigte Kommentare: {len(df_comments_clean)}")

## Google Drive Export

Google Drive Verbindung 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Export Funktion

In [ ]:
def export_to_drive(df_posts, df_comments, drive_path="/content/drive/My Drive/"):
    # Dateien erstellen
    posts_filename = f"{drive_path}gold_clean_posts.parquet"
    comments_filename = f"{drive_path}gold_clean_comments.parquet"
    
    # Export als Parquet-Dateien
    df_posts.to_parquet(posts_filename, index=False)
    df_comments.to_parquet(comments_filename, index=False)
    
    print(f"Posts exportiert nach: {posts_filename}")
    print(f"Kommentare exportiert nach: {comments_filename}")


Export Funnktion anwenden

In [ ]:
# Export der bereinigten Daten
export_to_drive(df_posts_clean, df_comments_clean)